In [1]:
%matplotlib inline

#### Neural Networks
使用`torch.nn`包来构建神经网络。
`nn`包依赖`autograd`包来定义模型并求导。一个`nn.Module`包含各个层和一个`forward(input)`方法，该方法返回`output`。

神经网络的典型训练过程如下：
1. 定义包含一些可学习的参数（或者叫权重）神经网络模型；
2. 在数据集上迭代；
3. 通过神经网络处理输入；
4. 计算损失（输入结果和正确值的差值大小）；
5. 将梯度反向传播回网络的参数；
6. 更新网络的参数，主要使用如下简单的更新原则：`weight = weight - learning_rate * gradient`

#### 定义网络

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 定义网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层1，输入通道1，输出通道6，卷积核5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 全连接层1
        self.fc2 = nn.Linear(120, 84)          # 全连接层2
        self.fc3 = nn.Linear(84, 10)           # 全连接层3

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # 卷积层1 + ReLU + 池化
        # if the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)    # 卷积层2 + ReLU + 池化
        x = x.view(-1, self.num_flat_features(x))  # 展平多维的卷积图成一维的向量
        x = F.relu(self.fc1(x))                     # 全连接层1 + ReLU
        x = F.relu(self.fc2(x))                     # 全连接层2 + ReLU
        x = self.fc3(x)                             # 全连接层3
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # 除批量大小外的所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
net = Net() # 创建网络实例
print(net) # 打印网络结构

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


在模型中必须要定义`forward`函数，`backward`函数（用来计算梯度）会被`autograd`自动创建。可以在`forward`函数中使用任何针对`Tensor`的操作。

`net.parameters()`返回可被学习的参数（权重）列表和值

In [3]:
params = list(net.parameters()) # 获取网络的所有参数
print(len(params)) # 打印参数的数量
print(params[0].size()) # 打印第一个参数的尺寸（卷积层1的权重）

10
torch.Size([6, 1, 5, 5])


测试随机输入32*32。

注：这个网络（LeNet）期望的输入大小是32 * 32，如果使用MINIST数据集来训练这个网络，请把图片大小重新调整到32*32。

In [4]:
input = torch.randn(1, 1, 32, 32) # 创建一个随机输入张量，尺寸为(1, 1, 32, 32)
out = net(input) # 前向传播
print(out) # 打印输出结果

tensor([[-0.0207,  0.0720,  0.0432,  0.0184,  0.0805, -0.0477, -0.1556,  0.0351,
         -0.0682,  0.0138]], grad_fn=<AddmmBackward0>)


将所有参数的梯度缓存清零，然后进行随机梯度的反向传播：

In [5]:
net.zero_grad() # 将所有参数的梯度缓存清零
out.backward(torch.randn(1, 10)) # 随机梯度的反向传播

### Note

`torch.nn`只支持小批量输入。整个`torch.nn`包都只支持小批量样本，而不支持单个样本。

例如，`nn.Conv2d`接受一个4维的张量，每一维分别是`sSamples * nChannels * Height * Width`（样本数 * 通道数 * 高 * 宽）。

如果你有单个样本，只需要使用`input.unsqueeze(0)`来添加其他维数。

#### 回顾：

- `torch.Tensor`：一个用过自动调用`backward()`实现支持自动梯度计算的多维数组，并且保存关于这个向量的梯度。
- `nn.Module`：神经网络模块。封装参数，移动到GPU上运行，导出，加载等。
- `nn.Parameter`：一种变量，当把它赋值给一个`Module`时，被自动地注册为一个参数。
- `autograd.Function`：实现一个自动求导操作地前向和反向定义，每个变量操作至少创建一个函数系欸但，每一个`Tensor`的操作都会创建一个接到创建`Tensor`和编码其历史的函数的`Function`节点。

#### 损失函数
一个损失函数接受一对（output, target）作为输入，计算一个值来估计网络的输出和目标值相差多少。
`nn`包中有很多不同的损失函数。`nn.MSELoss`是一个比较简单的损失函数，它计算输出和目标间的均方误差。

In [6]:
output = net(input)
target = torch.randn(10)  # 随机目标值
target = target.view(1, -1)  # 调整目标值的形状以匹配输出
criterion = nn.MSELoss()  # 定义均方误差损失函数

loss = criterion(output, target)
print(loss)

tensor(1.2995, grad_fn=<MseLossBackward0>)


现在，如果在反向过程中跟随`loss`，使用它的`.grad_fn`属性，将看到如下所示的计算图。

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

所以当我们调用`loss.backward()`时，整张计算图都会根据loss进行微分，而且图中所有设置为`requires_grad = True`的张量将会拥有一个随着梯度积累的`.grad`张量。

为了说明，让我们向后退几步：

In [7]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

#### 反向传播
调用`loss.backward()`获得反向传播的误差。

但是在调用前需要清楚已存在的梯度，否则梯度将被累加到已存在的梯度。

现在，我们将调用`loss.backward()`，并查看`conv1`层的偏差（bias）项在反向传播前后的梯度。

In [8]:
net.zero_grad() # 将所有参数的梯度缓存清零

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad) # 打印卷积层1的偏差项梯度（反向传播前）

loss.backward() # 反向传播

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad) # 打印卷积层1的偏差项梯度（反向传播后）

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0010, -0.0237,  0.0017,  0.0113, -0.0119,  0.0051])


#### 更新权重
在实践中最简单的权重更新规则时随机梯度下降（SGD）：

`weight = weight - learning_rate * gradient`

我们可以使用简单的Python代码实现这个规则：

In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

但是当使用神经网络时，想要使用各种不同的更新规则时，比如SGD，Nesterov-SGD, Adam, RMSPROP等，Pytorch中构建了一个包`torch.optim`实现了所有的这些规则。

In [10]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in your training loop:
optimizer.zero_grad()  # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # Does the update